# Element Parameter Detection

## Setup

In [ ]:
import numpy as np
import math
import random
import os
import torch
import sys
import copy

import importlib
import torch.nn as nn
from tqdm.notebook import tqdm_notebook as tqdm

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance

import ifcopenshell
from path import Path
import open3d as o3d

from src.elements import *
from src.ifc import *
from src. preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance
from src.icp import icp_finetuning
from src.chamfer import generate_elbow_cloud, get_chamfer_dist_single

In [ ]:
random.seed = 42

In [ ]:
#path = Path("ModelNet10")
#path = Path('/content/drive/MyDrive/ElementNet/')
path = Path('output/')
#savepath = '/content/drive/MyDrive/ElementNet/'
savepath = 'models/'

## Dataset

Transforms for training.

In [ ]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [ ]:
# cat= 'tee'
# train_ds = PointCloudData(path, category=cat, transform=train_transforms)
# valid_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)

# print('Train dataset size: ', len(train_ds))
# print('Valid dataset size: ', len(valid_ds))
# #print('Number of classes: ', len(train_ds.classes))
# print('Sample pointcloud shape: ', train_ds[0]['pointcloud'])
# print('Sample pointcloud label: ', train_ds[0]['properties'])
# print('Sample pointcloud label: ', train_ds[0]['properties'])
# #print('Class: ', inv_classes[train_ds[0]['category']])

# train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
# valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

## Model

## Training loop

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# input_size = 2048
# targets = train_ds.targets
# pointnet = PointNet(outputs=targets, input_size=input_size)
# pointnet.to(device);
# #pointnet = pointnet.double()

# optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.005)
# criterion = torch.nn.SmoothL1Loss()

# #train(pointnet, savepath, optimizer, criterion, device, targets, train_loader, valid_loader, epochs=50, save=True)

## Test

Analyze results statistically

POINTNET++

In [ ]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath('industrial-facility-relationships/'))
BASE_DIR = os.path.join(BASE_DIR, 'pointnet2')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))
path = Path('output/')
cat= 'elbow'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False

test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)
targets = test_ds.targets
testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=64)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = "pointnet2/log/classification/pointnet2_cls_ssg/"
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

    
checkpoint = torch.load(model_path + '/checkpoints/best_model.pth')
predictor.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
def test(model, loader, device, criterion):
    losses = []
    predictor = model.eval()
    cloud_list = []
    label_list = []
    output_list = []
    predictions_list = []
    inputs_list = []
    id_list = []
    parameter_id = 0
    tot = 0
    count = 0
    
    for j, data  in tqdm(enumerate(loader), total=len(loader)):
        inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)

        points, target, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device('cpu'))
        inputs = points.to(torch.device('cpu'))
        labels = target.to(torch.device('cpu'))
        ids = ids.to(torch.device('cpu'))
        print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
            print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
        count += outputs.size(0)
    print(tot/count)        

    return predictions_list, inputs_list, label_list, output_list, id_list, cloud_list

In [ ]:
with torch.no_grad():
    predictions_list, inputs_list, label_list, output_list, id_list, cloud_list = test(predictor.eval(), testDataLoader, device, test_criterion)
    
print(len(predictions_list), len(inputs_list))

ORIGINAL POINTNET

In [ ]:
# pointnet = PointNet(targets, input_size)
# #pointnet.load_state_dict(torch.load(savepath +'save_24.pth'))
# pointnet.load_state_dict(torch.load(savepath +'save_49.pth', map_location=torch.device('cpu')))
# pointnet.to(device);

# pointnet.eval();

In [ ]:
# # check regression
# cloud_list = []
# label_list = []
# output_list = []
# predictions_list = []
# inputs_list = []
# id_list = []
# parameter_id = 10
# with torch.no_grad():
#     tot = 0
#     count = 0
#     for data in valid_loader:
#         inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
#         outputs, __, __ = pointnet(inputs.transpose(1,2))
#         outputs = outputs.to(torch.device('cpu'))
#         inputs = inputs.to(torch.device('cpu'))
#         labels = labels.to(torch.device('cpu'))
#         ids = ids.to(torch.device('cpu'))
#         print(data['pointcloud'].size(), labels.size(), outputs.size())

#         for i in range(outputs.size(0)):
#             label_list.append(labels[i][parameter_id].item())
#             id_list.append(ids[i].item())
#             output_list.append(outputs[i][parameter_id].item())
#             predictions_list.append(outputs[i].numpy())
#             inputs_list.append(labels[i].numpy())
#             cloud_list.append(inputs[i].numpy())
#             ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
#             print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
#             tot += np.absolute(ratio)
#             #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
#         count += outputs.size(0)
#     print(tot/count)

In [ ]:
# calculate direction / position error
def error_calc(predictions_list, inputs_list, k, j, direction = True, use_direction = False, square_error=True):

    total_error = 0
    for i, pr in enumerate(predictions_list):
        if direction:
            if use_direction:
                pred = get_direction_from_trig(pr, k) 
                y = get_direction_from_trig(inputs_list[i], k)
            else:
                pred = get_direction_from_position(pr, k, j)
                y = get_direction_from_position(inputs_list[i], k, j)
            
        else:
            pred = [pr[k], pr[k+1], pr[k+2]]
            y = [inputs_list[i][k], inputs_list[i][k+1], inputs_list[i][k+2]]
        if square_error:
            total_error += sq_distance(pred[0], pred[1], pred[2], 
                                      y[0], y[1], y[2])
        else:
            #angle error
            total_error += math.degrees(math.acos(np.dot(pred, y)))
            #print('mag', math.degrees(math.acos(np.dot(pred, y))))
    print(total_error/len(predictions_list), len(predictions_list))


In [ ]:
k = 5
j = 4
direction = False
use_direction = True
square_error = True
error_calc(predictions_list, inputs_list, k, j, direction, use_direction, square_error)

In [ ]:
label_list, output_list, id_list = np.array(label_list), np.array(output_list), np.array(id_list)

In [ ]:
# ratio = np.absolute((label_list - output_list)/label_list)
# ratio_ind = ratio.argsort()
# id_list = id_list[ratio_ind]
# print(id_list[-10:-1])

# error_threshold = 0.1
# correct = ratio[np.where(ratio < error_threshold)]
# print(len(ratio), len(correct), len(correct)/len(ratio))


In [ ]:
cloud_id = 2
pcd_id = 24229  


In [ ]:
#plot error graph
fig = plt.figure(figsize=(12,4))
ratio_neg = (label_list - output_list)/label_list
n, bins, _ = plt.hist(ratio_neg, bins=np.arange(-2,2,0.02))
mid = 0.5*(bins[1:] + bins[:-1])
plt.errorbar(mid, n, yerr=0.01, fmt='none')

#### Visually analyse predictions and Fine tune with ICP

In [ ]:
def prepare_visualisation(pcd_id, cat, cloud_id, predictions_list):
    scale_factor = 1000
    pcd_path = "output/" + cat + "/test/" + str(pcd_id) + ".pcd"

    # load pcd and 'un-normalise'
    pcd_temp = o3d.io.read_point_cloud(pcd_path).points
    norm_pcd_temp = np.mean(pcd_temp, axis=0)
    pcd_temp -= norm_pcd_temp
    norm_factor = np.max(np.linalg.norm((pcd_temp), axis=1))
    points = (cloud_list[cloud_id].transpose(1, 0))*norm_factor + norm_pcd_temp
    pcd = o3d.utility.Vector3dVector(points)
    print("norm", norm_factor)

    preds = predictions_list[cloud_id].tolist()

    # scale predictions when necessary
    if cat == 'pipe':
        scalable_targets = [0,1]
    elif cat == 'elbow':  
        scalable_targets = [0,1,2]
    elif cat == 'tee':
        scalable_targets = [0,1,2,3]

    for j in scalable_targets:
        preds[j] = preds[j]*scale_factor*norm_factor
    
    preds = [float(pred) for pred in preds]
    return pcd, preds


In [ ]:
blueprint = 'data/sample.ifc'
temp_dir = "output/temp/"
target_dir = "output/tee/test/"

ifcConvert_executable = "scripts/./IfcConvert"
cloudCompare_executable = "cloudcompare.CloudCompare"
sample_size = 2048
threshold = 2

icp_correction = True
viewers, ifcs = [], []

for i in range(1,2):
    pcd_id = id_list[i].item()
    print(pcd_id) 
        
    pcd, preds = prepare_visualisation(pcd_id, cat, i, inputs_list)
    print(preds)
    
#     pcd2, input_preds = prepare_visualisation(pcd_id, cat, i, inputs_list)
#     indices_to_replace = [5, 6, 7]
#     for i in indices_to_replace:
#         preds[i] = input_preds[i]
    
    #try:
    if not icp_correction:
        viewer, ifc = visualize_predictions([pcd], cat, [preds], blueprint, visualize=True)

    else:
        viewer, ifc = icp_finetuning(pcd, pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                             ifcConvert_executable, cloudCompare_executable, sample_size, threshold)

    viewers.append(viewer) 
    ifcs.append(ifc)

    cloud = generate_elbow_cloud(preds)
    chamfer_distance = get_chamfer_dist_single(cloud, pcd)
    print("chamfer_distance", chamfer_distance)
#     o3d.io.write_point_cloud("axis.pcd", cloud)
#     cloud.points = pcd
#     o3d.io.write_point_cloud("pcd.pcd", cloud)

#     except:
#         print("ICP error", pcd_id)



In [ ]:
for v in viewers:
    print(v)

In [ ]:
# measures the point to point distance between predicted ifc and cloud (and vice-versa)
def confidence_metric(pcd, preds):
    

In [ ]:
# threshold = 0.02
# trans_init  = np.identity(4)

# source_dir = "output/predicted_ifc/"
# target_dir = "output/tee/test/"
# output_dir = "output/transformed/"

# source_files = os.listdir(source_dir)
# failed_count = 0

# for f in source_files:
#     if f.split('.')[-1] == 'pcd':
#         source = o3d.io.read_point_cloud(os.path.join(source_dir, f))
#         try:
#             target = o3d.io.read_point_cloud(os.path.join(target_dir, f))
#             save_path = os.path.join(output_dir, f)
#             icp(source, target, threshold, trans_init, save_path)
#         except:
#             failed_count += 1


## Mesh deformation

In [ ]:
print(torch. __version__)